#  1

- 데이터 세트의 환자가 당뇨병을 앓고 있는지 여부를 정확하게 예측하는 기계 학습 모델을 구축할 수 있습니까?

- 데이터 모델링에서 중요한것들은 모델을 아무리 바꿔도 스코어는 높아질 확률이 크지 않으므로
디테일 값을 바꿔주는 것 + 데이터 분석을 꼼꼼히 하는 것에 차이가 있음 

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # 분류 

In [2]:
data = pd.read_csv('diabetes.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
data.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
data[data == 0 ]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
764,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
766,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


- 연령별 당뇨여부
- 임신횟수의 편차가 큼으로 임신 유무로 나누어 보기

In [8]:
 #data[data['Pregnancies'] ==0]

In [9]:
#data.iloc[100]

In [10]:
#data['Pregnancies2'] = np.where(data['Pregnancies'] == 0, 0, 1)

In [11]:
#data['Pregnancies2'].value_counts()

In [12]:
#data.groupby(['Pregnancies2','Outcome'])['Age'].count()

 - 임신횟수는 0 가능하지만 
    Glucose, BloodPressure, SkinThickness, Insulin ,BMI 는 불가능 

 - 0 값을 median 이나 mean 으로 대체 필요 

- 나이에 따라 값을 다르게 매겨 주는게 당뇨에 영향을 주지 않을까?!

In [13]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [14]:
def ageRange(age) :
    if 21 <= age < 30 :
        return 1
    elif 30 <= age < 40 :
        return 2
    elif 40 <= age < 50 :
        return 3
    elif 50 <= age < 60 :
        return 4
    elif 60 <= age < 70 :
        return 5 
    else :
        return 6

In [15]:
# ageRange 나이대별로 만든거 저장 
data['Age2'] = data['Age'].apply(ageRange)

In [16]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Age2
0,6,148,72,35,0,33.6,0.627,50,1,4
1,1,85,66,29,0,26.6,0.351,31,0,2
2,8,183,64,0,0,23.3,0.672,32,1,2
3,1,89,66,23,94,28.1,0.167,21,0,1
4,0,137,40,35,168,43.1,2.288,33,1,2
...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,5
764,2,122,70,27,0,36.8,0.340,27,0,1
765,5,121,72,23,112,26.2,0.245,30,0,2
766,1,126,60,0,0,30.1,0.349,47,1,3


In [17]:
col_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

In [18]:
# np.where 사용해서 0값을 nan 값으로 나머지는 col그대로 대입 
for col in col_list :
    data[col] = np.where((data[col]==0), np.nan , data[col])
    #print(col, len(data[data[col] == 0 ]))

In [19]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Age2
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1,4
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0,2
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1,2
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,1
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1,2
...,...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0,5
764,2,122.0,70.0,27.0,NaN,36.8,0.340,27,0,1
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0,2
766,1,126.0,60.0,NaN,NaN,30.1,0.349,47,1,3


- nan 값으로 변경했으니 확인

In [20]:
data.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
Age2                          0
dtype: int64

- Glucose BloodPressure SkinThickness Insulin BMI = nan 값 / 체인지 

In [21]:
# 나이별 인슐린 중앙값
data.groupby('Age')['Insulin'].median()

Age
21     89.0
22     90.0
23    108.0
24    120.0
25    100.0
26     96.5
27    115.5
28    120.0
29    192.5
30    145.5
31    183.5
32     93.5
33    165.0
34    203.0
35     89.0
36    175.0
37    133.0
38    180.0
39    122.0
40    137.5
41    156.0
42    130.0
43    180.0
44    258.0
45    113.5
46    182.0
47    114.0
48     56.0
49    138.0
50    105.5
51    162.5
52    378.0
53    207.0
54    183.0
55    167.5
56    207.0
57    195.0
58    292.5
59    846.0
60    410.5
61    190.0
62      NaN
63    180.0
64      NaN
65      NaN
66      NaN
67      NaN
68      NaN
69      NaN
70      NaN
72      NaN
81     60.0
Name: Insulin, dtype: float64

In [22]:
# 위에도 있음 
col_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

In [23]:
for col in col_list :
    data[col].fillna(data.groupby('Age2')[col].transform('median'), inplace= True)

In [24]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Age2
0,6,148.0,72.0,35.0,192.0,33.6,0.627,50,1,4
1,1,85.0,66.0,29.0,140.0,26.6,0.351,31,0,2
2,8,183.0,64.0,32.0,140.0,23.3,0.672,32,1,2
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,1
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1,2
...,...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0,5
764,2,122.0,70.0,27.0,105.0,36.8,0.340,27,0,1
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0,2
766,1,126.0,60.0,31.0,131.0,30.1,0.349,47,1,3


In [25]:
index = data.groupby('Age2')['Glucose'].median().index

In [26]:
value = data.groupby('Age2')['Glucose'].median().values

In [27]:
print(index)
print(value)

Int64Index([1, 2, 3, 4, 5, 6], dtype='int64', name='Age2')
[109. 122. 123. 139. 136. 134.]


In [28]:
col_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

In [29]:
dict_tot = {}
for col in col_list :
    index = data.groupby('Age2')['Glucose'].median().index
    value = data.groupby('Age2')['Glucose'].median().values
    temp = {}
    for i in range(len(index)) :
        temp[index[i]] = value[i]
    dict_tot[col] = temp

In [30]:
dict_tot

{'Glucose': {1: 109.0, 2: 122.0, 3: 123.0, 4: 139.0, 5: 136.0, 6: 134.0},
 'BloodPressure': {1: 109.0, 2: 122.0, 3: 123.0, 4: 139.0, 5: 136.0, 6: 134.0},
 'SkinThickness': {1: 109.0, 2: 122.0, 3: 123.0, 4: 139.0, 5: 136.0, 6: 134.0},
 'Insulin': {1: 109.0, 2: 122.0, 3: 123.0, 4: 139.0, 5: 136.0, 6: 134.0},
 'BMI': {1: 109.0, 2: 122.0, 3: 123.0, 4: 139.0, 5: 136.0, 6: 134.0}}

In [31]:
with open('dict_tot.pkl','wb') as f :
    pickle.dump(dict_tot, f)

In [32]:
with open('dict_tot.pkl','rb') as f :
    load_dict = pickle.load(f)

In [33]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Age2
0,6,148.0,72.0,35.0,192.0,33.6,0.627,50,1,4
1,1,85.0,66.0,29.0,140.0,26.6,0.351,31,0,2
2,8,183.0,64.0,32.0,140.0,23.3,0.672,32,1,2
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0,1
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1,2
...,...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0,5
764,2,122.0,70.0,27.0,105.0,36.8,0.340,27,0,1
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0,2
766,1,126.0,60.0,31.0,131.0,30.1,0.349,47,1,3


- model make

In [34]:
model = SVC(kernel='linear',C=5,gamma=0.0001)

In [35]:
x = data.drop(['Outcome','Age2'], axis=1)
y = data['Outcome']

In [36]:
x_train, x_test, y_train, y_test = \
train_test_split(x,y, test_size=0.2, random_state = 0 , stratify = y )

In [37]:
model.fit(x_train, y_train)

SVC(C=5, gamma=0.0001, kernel='linear')

In [38]:
model.score(x_train, y_train)

0.7703583061889251

In [39]:
model.score(x_test, y_test)

0.7662337662337663

- best 결과값 찾아내기

In [40]:
data=[]
for c in range(1,5) : 
    for i in range(1, 10) : 
        gamma = 0.0001 * i
        model = SVC(C=c, gamma=gamma)
        model.fit(x_train,y_train)
        ret1 = model.score(x_train, y_train)
        ret2 = model.score(x_test, y_test)
        temp=[]
        temp.append(c)
        temp.append(gamma)
        temp.append(ret1)
        temp.append(ret2)
        data.append(temp)

In [41]:
df = pd.DataFrame(data, columns= ['C','gamma','train','test'])

In [42]:
df.head(5)

,C,gamma,train,test
0,1,0.0001,0.768730,0.766234
1,1,0.0002,0.773616,0.792208
2,1,0.0003,0.781759,0.792208
3,1,0.0004,0.786645,0.785714
4,1,0.0005,0.801303,0.779221


In [43]:
df.sort_values(by = ['test','train'], ascending = False).head(5)

,C,gamma,train,test
27,4,0.0001,0.781759,0.798701
18,3,0.0001,0.773616,0.798701
30,4,0.0004,0.828990,0.792208
29,4,0.0003,0.814332,0.792208
11,2,0.0003,0.794788,0.792208


- 모델 2개더 추가 해서 for 문 써서 한번에 돌리기

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
import warnings
warnings.filterwarnings('ignore')

In [45]:
model2 = LogisticRegression()
model3 = LogisticRegressionCV()

In [46]:
d = [(x_train, y_train), (x_test, y_test)]

In [47]:
def trainModel(model, d):
    for i, x in enumerate(d) : 
        if(i==0) :
            model.fit(x[0],x[1])
            print('train', model.score(x[0], x[1]))
        else :
            print('test', model.score(x[0], x[1]))   

In [48]:
trainModel(model , d)

train 0.8664495114006515
test 0.7857142857142857


In [49]:
trainModel(model2 , d)

train 0.7654723127035831
test 0.7597402597402597


In [50]:
trainModel(model3 , d)

train 0.7687296416938111
test 0.7727272727272727


---

- 모델 만들기

In [51]:
model4 = SVC() 

In [52]:
# 위에서 처리 했기에 주석 처리 
#x= data.drop('Outcome', axis=1)
#y = data['Outcome']

In [53]:
# 위에서 처리 했기에 주석 처리 
#x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0, stratify=y)

In [54]:
model4.fit(x_train, y_train)

SVC()

In [55]:
model4.score(x_train, y_train)

0.749185667752443

In [56]:
model4.score(x_test, y_test)

0.7792207792207793

- 결과 와 모델 저장 

In [57]:
import pickle

In [58]:
model4

SVC()

In [59]:
with open('model4.pkl','wb') as f :
    pickle.dump(model4, f)